In [12]:
import pandas as pd
from Bio import SeqIO
from local.constants import WORKSPACE_ROOT
from pathlib import Path

FIGURES = Path("./cache/figures")
FIGURES.mkdir(parents=True, exist_ok=True)

RUN = "hifi_meta"
CULTURE = "Ana"

# from
# /home/tony/workspace/projects/model_strains/main/nosco/01.basic_stats.ipynb

In [13]:
columns = "is_classified	sequenceID	taxID	query_length	nucleotide_identity_score	tax_rank	taxID:kmer_match_count".split()
# df_metabuli = pd.read_csv("../data/taxonomy/flye_raw/metabuli/taxonomy-metabuli/Ana_PS.tsv", names=columns, sep="\t")
# df_metabuli = pd.read_csv("../data/taxonomy/hifi_100x/metabuli/taxonomy-metabuli/Ana_PS.tsv", names=columns, sep="\t")
df_metabuli = pd.read_csv(f"../data/taxonomy/{RUN}/metabuli/taxonomy-metabuli/{CULTURE}_PS.tsv", names=columns, sep="\t")
print(df_metabuli.shape)
df_metabuli.head(2)

(6, 7)


,is_classified,sequenceID,taxID,query_length,nucleotide_identity_score,tax_rank,taxID:kmer_match_count
1,s0.ctg000001l,347672,45468,0.108813,subspecies,d_d__Bacteria;p_p__Cyanobacteriota;c_c__Cyanob...,347669:299 347671:48 347672:53 10023775:205 10...
1,s1.ctg000002c,347672,6953598,0.348317,subspecies,d_d__Bacteria;p_p__Cyanobacteriota;c_c__Cyanob...,347669:502509 347671:1349 347672:231 10023775:...


In [14]:
df_metabuli

,is_classified,sequenceID,taxID,query_length,nucleotide_identity_score,tax_rank,taxID:kmer_match_count
1,s0.ctg000001l,347672,45468,0.108813,subspecies,d_d__Bacteria;p_p__Cyanobacteriota;c_c__Cyanob...,347669:299 347671:48 347672:53 10023775:205 10...
1,s1.ctg000002c,347672,6953598,0.348317,subspecies,d_d__Bacteria;p_p__Cyanobacteriota;c_c__Cyanob...,347669:502509 347671:1349 347672:231 10023775:...
1,s2.ctg000003c,416977,3366651,0.666011,subspecies,d_d__Bacteria;p_p__Pseudomonadota;c_c__Alphapr...,10029558:638432
1,s3.ctg000004c,301646,4448838,0.353125,subspecies,d_d__Bacteria;p_p__Pseudomonadota;c_c__Alphapr...,301640:349597 301644:2 301646:12 10033521:4934...
1,s4.ctg000005c,392432,69468,0.172022,subspecies,d_d__Bacteria;p_p__Cyanobacteriota;c_c__Cyanob...,392430:813 392432:94 10055429:10 10055430:1 10...
1,s5.ctg000006c,384932,124176,0.127372,subspecies,d_d__Bacteria;p_p__Pseudomonadota;c_c__Alphapr...,384932:1713 10037265:482 10037266:261 10037267...


In [15]:
contig2mbtax = {}
for _, r in df_metabuli.iterrows():
    contig = r["is_classified"]
    tax_raw = r["tax_rank"]
    tax = tax_raw.split(";")
    phy = [x for x in tax if x[0] == "p"][0].split("_")[-1]
    gen = [x for x in tax if x[0] == "g"][0].split("_")[-1]
    contig2mbtax[contig] = phy, gen
contig2mbtax

{'s0.ctg000001l': ('Cyanobacteriota', 'Anabaenopsis'),
 's1.ctg000002c': ('Cyanobacteriota', 'Anabaenopsis'),
 's2.ctg000003c': ('Pseudomonadota', 'Erythrobacter'),
 's3.ctg000004c': ('Pseudomonadota', 'Allorhizobium'),
 's4.ctg000005c': ('Cyanobacteriota', 'Trichormus'),
 's5.ctg000006c': ('Pseudomonadota', 'Allorhizobium')}

In [16]:
contig2bin = {
    # flye raw
    "contig_1": "ANA",
    "contig_2": "ERY",
    "contig_3": "ANA",
    "contig_4": "RHI",
    "contig_5": "RHI",
    "contig_6": "ANA",

    # hifi_100x
    "ptg000001l": "p1",
    "ptg000002c": "ANA",
    "ptg000003c": "ERY",
    "ptg000004l": "RHI",
    "ptg000005c": "p2",
    "ptg000006l": "RHI",
    "ptg000007l": "RHI",

    # hifi meta
    "s0.ctg000001l":"p1",
    "s1.ctg000002c":"ANA",
    "s2.ctg000003c":"ERY",
    "s3.ctg000004c":"RHI",
    "s4.ctg000005c":"p2",
    "s5.ctg000006c":"p3",

}

In [17]:
_name = "hifi_meta"
# out_dir = Path("../data/assembly/flye_raw.ana_bins")
out_dir = Path(f"../data/assembly/{_name}.ana_bins")
out_dir.mkdir(exist_ok=True)
_bins = {} 
# for entry in SeqIO.parse("../data/assembly/flye_raw/sequences-flye_raw_assembly/Ana_PS.fna", "fasta"):
# for entry in SeqIO.parse("../data/assembly/hifi_100x/sequences-isolate_assembly/Ana_PS.fna", "fasta"):
_path = None
for _path in Path(f"../data/assembly/{_name}").glob("*/Ana_PS.fna"):
    break
print(_path)
assert _path is not None
for entry in SeqIO.parse(_path, "fasta"):
    _bin = contig2bin[entry.name]
    _bins[_bin] = _bins.get(_bin, [])+[entry]
for _bin, entries in _bins.items():
    out_file = out_dir/f"{_bin}.fna"
    # if out_file.exists(): continue
    print(_bin, [len(e.seq) for e in entries])
    with open(out_file, "w") as f:
        for e in entries:
            f.write(f">{e.name}"+"\n")
            f.write(f"{e.seq}"+"\n")

../data/assembly/hifi_meta/sequences-hifiasm_meta_assembly/Ana_PS.fna
p1 [45470]
ANA [6953602]
ERY [3366654]
RHI [4448840]
p2 [69472]
p3 [124178]


In [18]:
# checkm_dir = Path("../data/taxonomy/checkm.ana/taxonomy-checkm_stats")
# checkm_dir = Path("../data/taxonomy/hifi_100x/checkm.ana/taxonomy-checkm_stats")
checkm_dir = Path("../data/taxonomy/hifi_meta/checkm.ana/taxonomy-checkm_stats")
_dfs = [pd.read_csv(p, sep="\t") for p in checkm_dir.iterdir()]
df_checkm = pd.concat(_dfs)
bin2stats = {}
for _, r in df_checkm.iterrows():
    _bin = r["Bin Id"]
    _bin = _bin.split(".")[-1]
    comp = r["Completeness"]
    redu = r["Contamination"]
    bin2stats[_bin] = comp, redu
bin2stats

{'ERY': (99.48, 0.54),
 'RHI': (100.0, 0.71),
 'p1': (0.0, 0.0),
 'p3': (0.0, 0.0),
 'p2': (0.0, 0.0),
 'ANA': (99.48, 0.0)}

In [19]:
# df_cov = pd.read_csv("../data/assembly/flye_raw/sequences-assembly_per_contig_coverage/Ana_PS.csv", sep="\t")
# df_cov = pd.read_csv("../data/assembly/hifi_100x/sequences-assembly_per_contig_coverage/Ana_PS.csv", sep="\t")
df_cov = pd.read_csv("../data/assembly/hifi_meta/sequences-assembly_per_contig_coverage/Ana_PS.tsv", sep="\t")
print(df_cov.shape)
df_cov.head(2)

(6, 3)


,contig,fold_coverage,contig_length
0,s0.ctg000001l,457.985705,45470
1,s1.ctg000002c,313.776093,6953602


In [20]:
meta = []
for _, r in df_cov.iterrows():
    contig = r["contig"]
    cov = r["fold_coverage"]
    l = r["contig_length"]
    phy, gen = contig2mbtax[contig]
    _bin = contig2bin[contig]
    meta.append((contig, cov, l, phy, gen, _bin))

dfm = pd.DataFrame(meta, columns="contig, depth, length, mb_phylum, mb_genus, bin".split(", "))
dfm = dfm.sort_values(by=["mb_phylum", "bin", "mb_genus"], ascending=True)
dfm

,contig,depth,length,mb_phylum,mb_genus,bin
1,s1.ctg000002c,313.776093,6953602,Cyanobacteriota,Anabaenopsis,ANA
0,s0.ctg000001l,457.985705,45470,Cyanobacteriota,Anabaenopsis,p1
4,s4.ctg000005c,258.470117,69472,Cyanobacteriota,Trichormus,p2
2,s2.ctg000003c,99.277256,3366654,Pseudomonadota,Erythrobacter,ERY
3,s3.ctg000004c,17.136131,4448840,Pseudomonadota,Allorhizobium,RHI
5,s5.ctg000006c,14.079241,124178,Pseudomonadota,Allorhizobium,p3


In [168]:
from local.figures.template import BaseFigure, ApplyTemplate, go
from local.figures.colors import COLORS, Color, Palettes
import numpy as np

fig = BaseFigure(
    # shape=(2, 4),
    shape=(1, 4),
    vertical_spacing=0.05,
    # horizontal_spacing=0.0,
    # row_heights=[5, 5, 4, 6],
    row_heights=[5, 5, 5, 5],
    # column_widths=[5, 1],
)
_black = Color.Hex("#212121")

def _length(x):
    if x <1 :
        return f"{x*1000:0.1f} kb"
    else:
        return f"{x:0.1f} Mb"

x = [r["bin"] for _, r in dfm.iterrows()]

fig.add_trace(
    go.Bar(
        # x = [r["contig"].split("_")[-1] for _, r in dfm.iterrows()],
        x = x,
        y = [r["length"]/10**6 for _, r in dfm.iterrows()],
        # orientation="h",
        text = [_length(r["length"]/10**6) for _, r in dfm.iterrows()],
        textposition='outside',
        marker=dict(color=_black.color_value),
        showlegend=False,
    ),
    row=1, col=1,
)
_y = [r["depth"] for _, r in dfm.iterrows()]
fig.add_trace(
    go.Bar(
        # x = [r["contig"].split("_")[-1] for _, r in dfm.iterrows()],
        x = x,
        y = _y,
        text=[f"{v:0.0f}x" for v in _y],
        textposition='outside',
        # orientation="h",
        marker=dict(color=_black.color_value),
        showlegend=False,
    ),
    row=2, col=1
)

col_c, col_r = Palettes.PLOTLY[0].color_value, Palettes.PLOTLY[1].color_value
# col_c, col_r = Palettes.PLOTLY[2].color_value, Palettes.PLOTLY[1].color_value
for i, (_x, _y, _col, name) in enumerate([
    ([b for b, (c, r) in bin2stats.items()], [c for b, (c, r) in bin2stats.items()], col_c, "completeness"),
    ([b for b, (c, r) in bin2stats.items()], [r for b, (c, r) in bin2stats.items()], col_r, "redundancy"),
]):
    def sf(v, c):
        if c.startswith("p"):
            return ""
        if v<=1:
            return f"{v:0.2f}"
        if v <100:
            return f"{v:0.1f}"
        if v >=100: 
            return f"{v:0.0f}"
    
    fig.add_trace(
        go.Bar(
            y = _y,
            x = _x,
            # text = [v if not vx.startswith("p") else "" for v, vx in zip(y, x)],
            # textposition='outside',
            text = [f'<span style="color: {_col}">{sf(v, c)}</span>' for v, c in zip(_y, _x)],
            textposition='outside',
            # textangle=0,
            # orientation="h",
            name=name,
            marker=dict(color=_col),
            showlegend=True,
            # legendgroup="1",
            legend="legend",
        ),
        row=3, col=1
    )

y = [r["mb_genus"] for _, r in dfm.iterrows()]
z = [r["contig"].split("_")[-1] for _, r in dfm.iterrows()]
xyz = [(x, y, z) for x, y, z in zip(x, y, z)]
for suffix, name, symbol in [("c", "circular", "circle"), ("l", "linear", "line-ns")]:
    # x = [r["bin"] for _, r in dfm.iterrows()]
    fig.add_trace(
        go.Scatter(
            x = [x for x, y, z in xyz if z.endswith(suffix)],
            y = [y for x, y, z in xyz if z.endswith(suffix)],
            mode="markers",
            marker=dict(
                color=COLORS.TRANSPARENT, size=12, symbol=symbol,
                line=dict(color=_black.color_value, width=2),
            ),
            showlegend=True,
            legend="legend2",
            name=name,
        ),
        row=4, col=1
    )

vlines = False
_invis = dict(showticklabels=False, ticks=None, linecolor=COLORS.TRANSPARENT)
fig = ApplyTemplate(
    fig,
    # default_yaxis=dict(showticklabels=False),
    default_xaxis=_invis,
    axis={
        "1 1 y": dict(title="Length<br>(Mb)", range=[0, 10], title_standoff=105),
        "1 2 y": dict(title="Fold<br>coverage", range=[0, 600], dtick=300, title_standoff=95),
        "1 3 y": dict(
            title="CheckM<br>(%)",
            showgrid=False,
            # type="log",
            # dtick=2,
            # range=[-2, 3.7],
            range=[0, 135],
            dtick=50,
            # max
            title_standoff=95,
        ),
        "1 4 y": dict(
            showticklabels=True,
            title="Metabuli<br>genus",
            categoryorder="array", 
            categoryarray=['Anabaenopsis', 'Trichormus', 'Erythrobacter', 'Allorhizobium'][::-1],
            # categoryarray=['s1.ctg000002c', 's0.ctg000001l', 's2.ctg000003c', 's4.ctg000005c', 's3.ctg000004c', 's5.ctg000006c'],
            title_standoff=20
        ),
        "1 3 x": dict(showgrid=vlines),
        "1 4 x": dict(
            showgrid=vlines,
            showticklabels=True,
            ticks="outside",
            title="Contig",
            # categoryorder="array",
            # categoryarray="ANA, p1, p2, ERY, RHI, p3".split(", "),
            # categoryarray=['s1.ctg000002c', 's0.ctg000001l', 's2.ctg000003c', 's4.ctg000005c', 's3.ctg000004c', 's5.ctg000006c'],
            linecolor=COLORS.BLACK
        ),
        "2 4 y": _invis,
        # "2 4 x": dict(
        #     linecolor=COLORS.BLACK, showticklabels=True,
        #     ticks="outside", dtick=100,
        #     # range=[0, 150], 
        #     title=f'% <span style="color: {col_c}">Completeness</span>/<br><span style="color: {col_r}">Redundancy</span>',
        # ),
    },
    layout=dict(
        width=800, height=600,
        legend=dict(
            y=0.45,
            yanchor="top",
        ),
        legend2=dict(
            y=0.23,
            yanchor="top",
        )
    ),
)

# Ensure the legend follows the order of traces
fig.update_layout(legend_traceorder="grouped")
# fig.update_layout(uniformtext_minsize=6.9, uniformtext_mode='show')
fig.write_image(FIGURES/"contig_and_bin_stats.svg")
fig.show()

In [172]:
gtdb_dir = Path(f"../data/taxonomy/{RUN}/gtdbtk.{CULTURE.lower()}/taxonomy-gtdbtk/")
tables = []
for f in gtdb_dir.glob("*.tsv"):
    _df = pd.read_csv(f, sep="\t")
    tables.append(_df)

dft = pd.concat(tables)
for _, r in dft[["user_genome", "classification", "closest_placement_taxonomy", "closest_placement_reference", "closest_placement_ani", "classification_method", "note"]].iterrows():
    for k, v in dict(r).items():
        print(k, v, sep="\t")
    print()

user_genome	ERY
classification	d__Bacteria;p__Pseudomonadota;c__Alphaproteobacteria;o__Sphingomonadales;f__Sphingomonadaceae;g__Erythrobacter;s__
closest_placement_taxonomy	d__Bacteria;p__Pseudomonadota;c__Alphaproteobacteria;o__Sphingomonadales;f__Sphingomonadaceae;g__Erythrobacter;s__Erythrobacter sp002215495
closest_placement_reference	GCF_002215495.1
closest_placement_ani	94.32
classification_method	taxonomic classification defined by topology and ANI
note	classification based on placement in class-level tree

user_genome	RHI
classification	d__Bacteria;p__Pseudomonadota;c__Alphaproteobacteria;o__Rhizobiales;f__Rhizobiaceae;g__Allorhizobium;s__
closest_placement_taxonomy	nan
closest_placement_reference	nan
closest_placement_ani	nan
classification_method	taxonomic classification defined by topology and ANI
note	classification based on placement in class-level tree

user_genome	p1
classification	Unclassified
closest_placement_taxonomy	nan
closest_placement_reference	nan
closest_placem